In [16]:
from gs_quant.common import PayReceive, Currency
from gs_quant.instrument import IRSwap
from gs_quant.markets import PricingContext
from gs_quant.markets.portfolio import Portfolio
from gs_quant.risk import MarketDataPattern, MarketDataShock, MarketDataShockBasedScenario, MarketDataShockType
from gs_quant.session import Environment, GsSession

In [4]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

In [8]:
# define two swaps
swap1 = IRSwap(
    termination_date='10y', notional_currency=Currency.USD, pay_or_receive=PayReceive.Pay, fixed_rate=0.007, name='10y'
)
swap2 = IRSwap(
    termination_date='15y', notional_currency=Currency.USD, pay_or_receive=PayReceive.Pay, fixed_rate=0.007, name='15y'
)

In [9]:
# create a couple of scenarios

# 1bp Spot Shock
ir_1bp_scenario = MarketDataShockBasedScenario(
    shocks={MarketDataPattern('IR', 'USD'): MarketDataShock(shock_type=MarketDataShockType.Absolute, value=1e-4)}
)

# 2bp Spot Shock
ir_2bp_scenario = MarketDataShockBasedScenario(
    shocks={MarketDataPattern('IR', 'USD'): MarketDataShock(shock_type=MarketDataShockType.Absolute, value=1e-4)}
)

In [12]:
# if we want to apply one scenario to many trades it is best to join those trades into a portfolio and send this in one calc

portfolio = Portfolio([swap1, swap2], name='portfolio')

# note you can append (and pop) trades into a portfolio but this is generally less efficient than just creating with list comprehension
# also portfolios can themselves contain portfolios

In [13]:
# many trades with one scenario

with ir_1bp_scenario:
    prices = portfolio.price()

prices.to_frame()

H:\code\gs_quant\gs_quant\risk\results.py:80: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_df = df.pivot_table(values=values, index=index, columns=columns, aggfunc=aggfunc)


risk_measure,Price
instrument_name,
10y,2.295177e+07
15y,3.253489e+07


In [15]:
# now I want to apply 1 scenario to 1 trade and the other scenario to the 2nd trade
# you could do this sequentially and that will work but be slow.  Better to nest the scenarios in a overarching context

with PricingContext():  # this context just wraps the calcs so that they will be run in parallel
    with ir_1bp_scenario:
        p1 = swap1.price()
    with ir_2bp_scenario:
        p2 = swap2.price()

print(p1.result())
print(p2.result())

22951766.87812186
32534888.041928925
